In [ ]:
!pip install dlib

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Set to 0 if want to use 1st GPU, 1 if want to use 2nd GPU, etc.

In [ ]:
from pathlib import Path
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import shutil
import time
import dlib
from random import shuffle

In [ ]:
# https://github.com/davisking/dlib-models/raw/master/mmod_human_face_detector.dat.bz2
detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

In [ ]:
# Video Dataset should be in structure:
"""
DatasetName
|
|- train
|    |- real
|    |- fake
|
|- val
|   |- real
|   |- fake
"""

In [ ]:
videos_path = 'PATH CONTAINING THE VIDEO DATASET/'
output_path = 'PATH WHERE THE FACE DATASET SHOULD GO/'

In [ ]:
for split in ['train/','val/']:
    for label in ['real/','fake/']:
        print(videos_path+split+label+': '+str(len([f for f in os.listdir(videos_path+split+label) if 'mp4' in f])))

In [ ]:
for split in ['train/','val/']:
    for label in ['real/','fake/']:
        Path(output_path+split+label).mkdir(parents=True, exist_ok=True)

In [ ]:
split = 'train/'
label = 'real/'
vidpath = videos_path + split + label
folderpath = output_path + split + label
videos = sorted(os.listdir(vidpath))
shuffle(videos)
videos = videos[:len([f for f in os.listdir(videos_path+split+'real/') if 'mp4' in f])]
req_frames = 30
scale = 1.3
req_size = 200

In [ ]:
logs_dict = {}

In [ ]:
for video in tqdm(videos):
    facepath = folderpath + video[:-4] + '/'
    Path(facepath).mkdir(parents=True,exist_ok=True)
    video_capture = cv2.VideoCapture(vidpath + video)
    count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    more_than_one = 0
    no_faces = 0
    out_of_frame = 0
    done_frames = 0
    img_size = -1
    frame_number = 0
    while True:
        res, frame = video_capture.read()
        if not res:
            break
        faceRects = detector(frame[:,:,::-1], 0)
        if (len(faceRects)==0):
            no_faces += 1
            continue
        if (len(faceRects)>1):
            more_than_one += 1
            continue
        x1 = faceRects[0].rect.left()
        y1 = faceRects[0].rect.top()
        x2 = faceRects[0].rect.right()
        y2 = faceRects[0].rect.bottom()
        center = ((x1 + x2) // 2, (y1 + y2) // 2)
        length = max(x2 - x1, y2 - y1)
        x1 = center[0] - int(length*scale/2)
        x2 = center[0] + int(length*scale/2)
        y1 = center[1] - int(length*scale/2)
        y2 = center[1] + int(length*scale/2)
        if x1 < 0 or y1 < 0 or x2 > frame.shape[1] or y2 > frame.shape[0]:
            out_of_frame += 1
            continue
        final=frame[y1:y2,x1:x2,::-1]
        # Either resize images now or resize them during training
        # final = cv2.resize(final,(req_size,req_size),interpolation = cv2.INTER_CUBIC)
        plt.imsave(facepath+str(frame_number).zfill(2)+'.jpg',final)
        frame_number += 1
        done_frames += 1
        if done_frames == req_frames:
            img_size = y2-y1
            break
    logs_dict[video] = [more_than_one, no_faces, out_of_frame, img_size, done_frames]
    video_capture.release()

In [ ]:
import json

In [ ]:
with open('dlib_30faces_{}{}_logs.json'.format(split[:-1],label[:-1]), 'w') as fp:
    json.dump(logs_dict, fp)